In [102]:
#Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [58]:
#scraping the news
news_url='https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response=requests.get(news_url)
soup=BeautifulSoup(response.text,'html.parser')
news_title=soup.find_all('div', class_='rollover_description_inner')[0].text
news_title=news_title.split('\n')
news_title=news_title[1]

news_p=soup.find_all('div', class_='content_title')[0].text
news_p=news_p.split('\n')
news_p=news_p[2]
news_p

"The Launch Is Approaching for NASA's Next Mars Rover, Perseverance"

In [105]:
#scraping featured image
browser= webdriver.Chrome('chromedriver.exe')
browser.get('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')
button=browser.find_element_by_link_text("FULL IMAGE")
button.click()
pictures=browser.find_elements_by_tag_name('img')
time.sleep(3)
picture=browser.find_element_by_class_name("fancybox-image")
all_links=[]
featured_image_url=[]
for image in pictures:
    all_links.append(image.get_attribute('src'))
#for link in all_links:
#    if link[-3:]=="jpg" and "mediumsize" in link:
#        print(link)
            

In [106]:
all_links

['https://www.jpl.nasa.gov/assets/images/logo_nasa_trio_black@2x.png',
 'https://www.jpl.nasa.gov/assets/images/logo_nasa_trio_black@2x.png',
 'https://www.jpl.nasa.gov/assets/images/overlay-arrow.png',
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24002-640x350.jpg',
 'https://www.jpl.nasa.gov/assets/images/overlay-arrow.png',
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24001-640x350.jpg',
 'https://www.jpl.nasa.gov/assets/images/overlay-arrow.png',
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24000-640x350.jpg',
 'https://www.jpl.nasa.gov/assets/images/overlay-arrow.png',
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23999-640x350.jpg',
 'https://www.jpl.nasa.gov/assets/images/overlay-arrow.png',
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23998-640x350.jpg',
 'https://www.jpl.nasa.gov/assets/images/overlay-arrow.png',
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23997-640x350.jpg',
 'https://www.jpl.

In [72]:
button=browser.find_element_by_link_text("FULL IMAGE")

AttributeError: 'WebDriver' object has no attribute 'find_element_by_link_text'

In [107]:
picture=browser.find_element_by_class_name("fancybox-image")

In [100]:
picture.find_elements_by

<selenium.webdriver.remote.webelement.WebElement (session="6e159a2109e546cafd26362397fb56d3", element="0.11334985571068801-76")>

In [86]:
pictures=browser.find_elements_by_tag_name('img')

In [90]:
for picture in pictures:
    print(picture.get_attribute('src'))

https://www.jpl.nasa.gov/assets/images/logo_nasa_trio_black@2x.png
https://www.jpl.nasa.gov/assets/images/logo_nasa_trio_black@2x.png
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24002-640x350.jpg
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24001-640x350.jpg
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24000-640x350.jpg
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23999-640x350.jpg
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23998-640x350.jpg
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23997-640x350.jpg
https://www.jpl.nasa.gov/assets/images/overlay-arrow.png
https://www.jpl.n